In [1]:
from lxml import html

from bs4 import BeautifulSoup
import time
import datetime

import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

import json
import os

from apscheduler.schedulers.background import BackgroundScheduler
import sys

import helper

In [2]:
IMPLICIT_WAIT_TIME = 10
EXPLICIT_WAIT_TIME = 10
WINDOWS_GRID_X = 3
WINDOWS_GRID_Y = 2
# SCREEN_WIDTH = 1920
# SCREEN_HEIGHT = 1080
SCREEN_WIDTH = 2560
SCREEN_HEIGHT = 1440
# SCREEN_WIDTH = 2048
# SCREEN_HEIGHT = 1152
# SCREEN_WIDTH = 1920
# SCREEN_HEIGHT = 1200
# SCREEN_WIDTH = 3200
# SCREEN_HEIGHT = 1800
WINDOW_WIDTH = SCREEN_WIDTH / WINDOWS_GRID_X
WINDOW_HEIGHT = SCREEN_HEIGHT / WINDOWS_GRID_Y

TIME_INTERVAL_SIGNIN_BOOKING = 180 #3 minutes
DELAY_TIME_BOOKING = 0.45

IMPLICIT_WAIT_TIME = 10
EXPLICIT_WAIT_TIME = 10

In [3]:
NO_WINDOW = False

In [4]:
course_schedule = []

In [5]:
from selenium.common.exceptions import NoSuchElementException        
def check_exists_by_xpath(xpath):
    try:
        driver.find_element("xpath",xpath)
    except NoSuchElementException:
        return False
    return True

def sync_get_element_by_xpath(driver, wait, xpath):
    wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    return driver.find_element("xpath",xpath)

In [6]:
def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)
        
def sign_in(driver, wait):
    
    username = driver.find_element("id",'ctl00_cphContents_txtUsername')
    password = driver.find_element("id",'ctl00_cphContents_txtPassword')
    
    send_keys(username, "DN20092360")
    send_keys(password, "jj1216")
#     username.send_keys("DN23035242")
#     password.send_keys("NTW3BSFH")

    wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "submit-button")))

    sign_in_btn = driver.find_element(By.CLASS_NAME,"submit-button")
    sign_in_btn.click()
    
    
def click_book_this_class_now(driver, wait):
    
    click_btn = None
    try:
        click_btn = sync_get_element_by_xpath(driver, wait, "//a[contains(text(), '現在就預定此課')]")
    except TimeoutException:
        message = driver.find_element("xpath","//div[@class='class-info']/div[2]/div[2]/input[2]").text
        print("driver: {} , {}".format(driver, message))
    
    # click_btn = driver.find_element("xpath","//a[contains(text(), '現在就預定此課')]") 
    # try:
    #     click_btn = driver.find_element("id",'ctl00_cphContents_btnBook')
    #     click_btn.click()
    # except NoSuchElementException:
    #     print("Error: click_book_this_class_now -- cannot find the book now button")
    
    print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
        return True
    else:
        return False


In [7]:
def scheduled_sign_in(driver, wait, class_to_be_booked):
    print("scheduled_sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    time.sleep(1)
    sign_in(driver, wait)

    
def reserve_class(driver, wait, class_to_be_booked):
    
    print("reserve_class: ({})\n".format(class_to_be_booked['time']))
    
    book_url = class_to_be_booked['href']
    class_name = class_to_be_booked['name']
    
    driver.get(book_url)

    click_book_this_class_now(driver, wait)

In [8]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [9]:
def create_new_driver():
    #setup option for chrome profile
    chrome_options = Options()
    
    ## NEVER USE THE FOLLOWING OPTION SINCE IT WILL MAKE SUPPOSEDLY DIFFERENT WINDOWS FOR DIFFERENT CLASS MIXED IN SOME WAY!!!!
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    ## the following option is to solve the error if using the above cause a problem
    # chrome_options.add_argument("--remote-debugging-port=9222") 
    
    prefs = {'profile.default_content_setting_values.automatic_downloads': 1}
    chrome_options.add_experimental_option("prefs", prefs)

    if NO_WINDOW:
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-gpu')
        print("LOG: NO WINDOW")
    
    #start web driver
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)
    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    
    return driver, wait


In [10]:
# load schedule from file
with open("./class_schedules/06 十一月 - 12 十一月 2023.json", "r") as read_file:
    course_schedule = json.load(read_file)

In [11]:
    # setup classes to be booked
    classes_to_be_booked = []

    ##---------------------------------------------------------------
     Day_string = "10"

    c = helper.search_courses_by_date(Day_string, course_schedule)
    # c = helper.search_courses_by_name('Aeri', c)
    c = helper.search_courses_by_teacher('Una', c)
    # c = helper.search_courses_by_time('pm', c)

    classes_to_be_booked.extend(c)
    ##---------------------------------------------------------------

    classes_to_be_booked

[{'room': '敦南 6 教室',
  'date': '08 11月 周三',
  'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1242711',
  'name': 'Aerial Yoga 空中瑜珈 (低空)',
  'teacher': 'Shannon',
  'time': '08:45 - 09:45 下午'},
 {'room': '敦南 4 教室',
  'date': '09 11月 周四',
  'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1242677',
  'name': 'Yoga Tone',
  'teacher': 'Una',
  'time': '06:15 - 07:15 下午'},
 {'room': '敦南 4 教室',
  'date': '09 11月 周四',
  'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1242678',
  'name': 'Rocket Yoga 火箭瑜珈',
  'teacher': 'Una',
  'time': '07:30 - 09:00 下午'},
 {'room': '敦南 6 教室',
  'date': '10 11月 周五',
  'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=1242720',
  'name': 'Yoga Tone 2',
  'teacher': 'Una',
  'time': '02:00 - 03:00 下午'},
 {'room': '敦南 6 教室',
  'date': '10 11月 周五',
  'href': 'https://www2.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id

In [12]:
# retrieve class date and time, convert it in python's datetime structure then schedule a process to book!

drivers = []

scheduler = BackgroundScheduler()

for idx,class_to_be_booked in enumerate(classes_to_be_booked):

    #retrieve class date and time, convert it in python's datetime structure
    
    date_comp = class_to_be_booked['date'].split(" ")
    date_comp[1] = date_comp[1].replace("月","")
    date_comp.pop()
    month = int(date_comp[1])
    day = int(date_comp[0])
    # month,day

    time_comp = class_to_be_booked['time'].split(" ")
    hour = time_comp[0].split(":")[0]
    minute = time_comp[0].split(":")[1]
    hour = int(hour)
    minute = int(minute)

    if time_comp[3] == "下午":
        hour = hour+12

        # deal with pm notation but at noon 12 pm special case
        if hour == 24:
            hour = 12
    # hour,minute

    from datetime import date
    today = date.today()
    year = int(today.strftime("%Y"))
    class_datetime = datetime.datetime(year, month, day, hour, minute, 0)
    book_datetime = class_datetime - datetime.timedelta(seconds=72*60*60)
    print("book_datetime= {}".format(book_datetime))

    # setup web driver and scheduled process for each class

    driver, wait = create_new_driver()
    drivers.append(driver)

    x = WINDOW_WIDTH * (idx % WINDOWS_GRID_X)
    y = WINDOW_HEIGHT * int(idx / WINDOWS_GRID_X)
    print("x={}, y={}".format(x,y))
    driver.set_window_position(x, y, windowHandle='current')
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)

    now = datetime.datetime.now()

    if (book_datetime - datetime.timedelta(seconds=TIME_INTERVAL_SIGNIN_BOOKING)) < now:
        print("sign in and book immediately")
        
        signin_datetime = now + datetime.timedelta(seconds=4+idx)
        book_datetime = now + datetime.timedelta(seconds=10+idx)
        if book_datetime < now:
            print("too late for booking")
            continue

    else:
        signin_datetime = book_datetime - datetime.timedelta(seconds=TIME_INTERVAL_SIGNIN_BOOKING)
    
    book_datetime = book_datetime + datetime.timedelta(seconds=DELAY_TIME_BOOKING)
    
    print("idx: ", idx, "scheduled signin datetime: ", signin_datetime)
    print("idx: ", idx, "scheduled book datetime: ", book_datetime)
    
    scheduler.add_job(scheduled_sign_in, 'date', run_date=signin_datetime, args=[driver, wait, class_to_be_booked])
    scheduler.add_job(reserve_class, 'date', run_date= book_datetime, args=[driver, wait, class_to_be_booked])

scheduler.start()
drivers

book_datetime= 2023-11-05 20:45:00
x=0.0, y=0.0
idx:  0 scheduled signin datetime:  2023-11-05 20:42:00
idx:  0 scheduled book datetime:  2023-11-05 20:45:00.450000
book_datetime= 2023-11-06 18:15:00
x=853.3333333333334, y=0.0
idx:  1 scheduled signin datetime:  2023-11-06 18:12:00
idx:  1 scheduled book datetime:  2023-11-06 18:15:00.450000
book_datetime= 2023-11-06 19:30:00
x=1706.6666666666667, y=0.0
idx:  2 scheduled signin datetime:  2023-11-06 19:27:00
idx:  2 scheduled book datetime:  2023-11-06 19:30:00.450000
book_datetime= 2023-11-07 14:00:00
x=0.0, y=720.0
idx:  3 scheduled signin datetime:  2023-11-07 13:57:00
idx:  3 scheduled book datetime:  2023-11-07 14:00:00.450000
book_datetime= 2023-11-07 15:30:00
x=853.3333333333334, y=720.0
idx:  4 scheduled signin datetime:  2023-11-07 15:27:00
idx:  4 scheduled book datetime:  2023-11-07 15:30:00.450000
book_datetime= 2023-11-07 12:45:00
x=1706.6666666666667, y=720.0
idx:  5 scheduled signin datetime:  2023-11-07 12:42:00
idx:  5

[<selenium.webdriver.chrome.webdriver.WebDriver (session="dd33b84c92da26e5f6edad266df6c865")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="f8abc45d3c7fc1804775190f439937a8")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="72892288ee47e798d8ba38bae6c5dd12")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="53a5f1c82e79840fbed06bd8bf72ccaf")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="fb8c70d522599176905c24637ccc3831")>,
 <selenium.webdriver.chrome.webdriver.WebDriver (session="c6913eb2d4489ed6a95a4dc4c5ce3358")>]

In [13]:
# elms = drivers[0].find_element("xpath","//div[@class='class-info']/div[2]/div[2]").get_attribute("innerHTML").split(">")
# # len(elms)
# # elms[6]
# elms


scheduled_sign_in(<selenium.webdriver.chrome.webdriver.WebDriver (session="dd33b84c92da26e5f6edad266df6c865")>,<selenium.webdriver.support.wait.WebDriverWait (session="dd33b84c92da26e5f6edad266df6c865")>)
reserve_class: (08:45 - 09:45 下午)

click_btn:  <selenium.webdriver.remote.webelement.WebElement (session="dd33b84c92da26e5f6edad266df6c865", element="1B32ADC19CA9788F88FD2AC3508F3EA6_element_11")>
scheduled_sign_in(<selenium.webdriver.chrome.webdriver.WebDriver (session="f8abc45d3c7fc1804775190f439937a8")>,<selenium.webdriver.support.wait.WebDriverWait (session="f8abc45d3c7fc1804775190f439937a8")>)
reserve_class: (06:15 - 07:15 下午)

click_btn:  <selenium.webdriver.remote.webelement.WebElement (session="f8abc45d3c7fc1804775190f439937a8", element="11C6C28523F2E3E3ED19BF2972C21E7A_element_11")>
